<a href="https://colab.research.google.com/github/Pickles91/DCAC/blob/main/Prostate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. GPU acceleration is available?



In [1]:
import torch

# Check whether GPU-accelerated computing is available
if torch.cuda.is_available():
    print("------------------------------")
    print("GPU acceleration is available.")
    print("------------------------------")
else:
    print("------------------------------------------------------------------------------------")
    print("Error: GPU acceleration is not available. Please enable GPU in the Runtime settings.")
    print("------------------------------------------------------------------------------------")

------------------------------
GPU acceleration is available.
------------------------------


# 2. Installing nnU-Net

In [2]:
# install nnunet
!pip install nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.1/184.1 kB 971.9 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 12.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.

**Note**: you do not have to restart the Runtime even when the following error appears, this is simply due to a reinstallation of a package - so no worries.


```
WARNING: The following packages were previously imported in this runtime:

[argparse]

You must restart the runtime in order to use newly installed versions.
```

In [1]:
# check if nnunet can be imported
import nnunetv2

# 3. Import Packages for Google Colab

In [2]:
# Import basic packages
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

# 4. Connect Google Colab with GoogleDrive

In [3]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## 5. Creating directories for nnUNet data and models


In [11]:
# Creating directories for nnUNet data and models
!mkdir -p "/content/drive/My Drive/Colab Notebooks/nnUNet_raw"
!mkdir -p "/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed"
!mkdir -p "/content/drive/MyDrive/Colab Notebooks/nnUNet_trained_models"
!mkdir -p "/content/drive/My Drive/Colab Notebooks/nnUNet_results"


# Changing directory to nnUNet_raw_data to download and extract data
%cd "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw"

# Downloading and extracting the data for Task05_Prostate
!gdown https://drive.google.com/uc?id=1Ff7c21UksxyT4JfETjaarmuKEjdqe1-a
!tar -xvf Task05_Prostate.tar
!rm Task05_Prostate.tar

# Changing directory back to the parent directory
%cd ..

/content/drive/MyDrive/Colab Notebooks/nnUNet_raw
Downloading...
From (original): https://drive.google.com/uc?id=1Ff7c21UksxyT4JfETjaarmuKEjdqe1-a
From (redirected): https://drive.google.com/uc?id=1Ff7c21UksxyT4JfETjaarmuKEjdqe1-a&confirm=t&uuid=9780ac8f-1756-417d-92ea-c063d38816c2
To: /content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Task05_Prostate.tar
100% 240M/240M [00:01<00:00, 209MB/s]
Task05_Prostate/._dataset.json
Task05_Prostate/dataset.json
Task05_Prostate/._imagesTr
Task05_Prostate/imagesTr/
Task05_Prostate/imagesTr/prostate_16.nii.gz
Task05_Prostate/imagesTr/._prostate_04.nii.gz
Task05_Prostate/imagesTr/prostate_04.nii.gz
Task05_Prostate/imagesTr/prostate_32.nii.gz
Task05_Prostate/imagesTr/prostate_20.nii.gz
Task05_Prostate/imagesTr/prostate_43.nii.gz
Task05_Prostate/imagesTr/prostate_18.nii.gz
Task05_Prostate/imagesTr/prostate_06.nii.gz
Task05_Prostate/imagesTr/prostate_14.nii.gz
Task05_Prostate/imagesTr/prostate_41.nii.gz
Task05_Prostate/imagesTr/prostate_34.nii.gz
Task05

In [12]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

print("Current Working Directory {}".format(os.getcwd()))
print(" ")
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"),
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results"),
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path

def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print(" ")
print("If No Error Occured Continue Forward. =)")

Current Working Directory /content/drive/MyDrive/Colab Notebooks
 
/content/drive/My Drive/Colab Notebooks/nnUNet_raw exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_results exists.
 
If No Error Occured Continue Forward. =)


In [13]:
# Install virtualenv package, create a virtual environment named 'myenv',
# activate the virtual environment, and install a package named 'package_name'.
!pip install virtualenv
!virtualenv myenv
!source myenv/bin/activate; pip install package_name

created virtual environment CPython3.10.12.final.0-64 in 11534ms
  creator CPython3Posix(dest=/content/drive/My Drive/Colab Notebooks/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: package_name==0.1, pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


### Dataset Conversion
The Decathlon datasets are 4D nifti files, for nnU-Net they have to be converted to 3D nifti files.

For more information about dataset conversion see: [nnU-Net Dataset Formatting Instructions](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_format.md)

In [14]:
!nnUNetv2_convert_MSD_dataset -i "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Task05_Prostate"

## Extracting Rule Based Parameters
This will preprocess the dataset to allow fast training and saves it into the "nnUNet_preprocessed" folder.
Further rule based parameters will be extracted in the planning step.

In [15]:
# Prepare the Execution of nnU-Net for Dataset 5
!nnUNetv2_plan_and_preprocess -d 5

Fingerprint extraction...
Dataset005_Prostate
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 32/32 [00:32<00:00,  1.03s/it]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [ 20. 320. 319.], 3d_lowres: [20, 320, 319]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 32, 'patch_size': (320, 320), 'median_image_size_in_voxels': array([320., 319.]), 'spacing': array([0.625, 0.625]), 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [False, False], 'resampling_fn_data': 'resample_data_

### Training nnU-Net
here we will train a 3D nnU-Net on Full Resolution for 2 epochs.

To run a normal training use: ```nnUNetTrainer``` instead of ```nnUNetTrainer_1epoch```

In [16]:
# train the 3d nnUnet on the Full Resolution with Dataset 5 and Cross Validation Split 0
!nnUNetv2_train 5 3d_fullres 0 -tr nnUNetTrainer_1epoch

# Training for 2 iterations will take about 11 minutes


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-04-23 02:35:16.160782: do_dummy_2d_data_aug: True
2024-04-23 02:35:16.166652: Creating new 5-fold cross-validation split...
2024-04-23 02:35:16.175877: Desired fold for training: 0
2024-04-23 02:35:16.179792: This split has 25 training and 7 validation cases.
/usr/lib/python3.10/multiproces

## Inference with Pre-Trained Models on Decathlon Prostate Dataset (WIP)
Here we will use a pretrained model on the Prostate Dataset and visualize the results

### Downloading Pretrained Model
First Download the pretrained models and validate that everything works correctly

In [17]:
# Download the Pretrained Model for the Prostate Dataset (Decathlon Dataset)
os.chdir(path_dict["nnUNet_results"])
!gdown 'https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy'
!nnUNetv2_install_pretrained_model_from_zip nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip
!rm 'nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip'
os.chdir(base_dir)

# takes roughly 9 minutes

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy
From (redirected): https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy&confirm=t&uuid=b41d1763-65df-4370-a893-7564ae863e2f
To: /content/drive/My Drive/Colab Notebooks/nnUNet_results/nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip
100% 1.66G/1.66G [00:18<00:00, 88.2MB/s]


In [18]:
# Preprocess Prostate Dataset
!nnUNetv2_convert_MSD_dataset -i "${RAW_DATA_PATH}/Task05_Prostate"

Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_convert_MSD_dataset", line 8, in <module>
    sys.exit(entry_point())
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/dataset_conversion/convert_MSD_dataset.py", line 128, in entry_point
    convert_msd_dataset(args.i, args.overwrite_id, args.np)
  File "/usr/local/lib/python3.10/dist-packages/nnunetv2/dataset_conversion/convert_MSD_dataset.py", line 62, in convert_msd_dataset
    assert len(existing_datasets) == 0, f"Target dataset id {target_id} is already taken, please consider changing " \
AssertionError: Target dataset id 5 is already taken, please consider changing it using overwrite_target_id. Conflicting dataset: ['Dataset005_Prostate'] (check nnUNet_results, nnUNet_preprocessed and nnUNet_raw!)


### Inference on the Prostate Dataset
Use 3d_fullres nnUnet (-c) trained on the test set of dataset 5 (-d 5) and save the outputs

In [19]:
# use fully trained nnU-Net to make predictions on data
!nnUNetv2_predict -i "${nnUNet_raw}/Dataset005_Prostate/imagesTs/" -o "${nnUNet_results}/Dataset005_Prostate/predTs/" -d 5 -c 3d_fullres


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/usr/local/lib/python3.10/dist-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
There are 16 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start c

## Visualization of Data and Segmentations
Download files from Google Drive:

Images from: ```${nnUNet_raw}/Dataset005_Prostate/imagesTs/```

Segmentations from: ```${RESULTS_FOLDER}/Dataset005_Prostate/predTs/"```


After downloading these files you can visualize them with any volumetric visualization program.
For this we would advise to use [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK)) which already has some great [tutorials](https://www.mitk.org/wiki/Tutorials).


Note:
- If you have not already downloaded it, here is the [MITK Download Link](https://www.mitk.org/wiki/Downloads)

# 7. How to train and use nnU-Net on a new Dataset?

Goal of the next Steps:
- How can you Implement nnU-Net for a new Dataset?
  - Example using the GM Spinal Cord Segmentation Challenge Dataset
    - [Data & Task Explanation](https://www.sciencedirect.com/science/article/pii/S1053811917302185)
    - [Data Download Link](http://cmictig.cs.ucl.ac.uk/niftyweb/challenge/)
- How to store the Data for use with nnU-Net?
  - General Information can be found [here](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_conversion.md)
- How to create the Data Fingerprint?
- How to create the Pipeline Fingerprint based on Rules?

This part of the Tutorial based on:

The [GoogleColab Notebook](https://github.com/prateekgupta891/nnUNet/blob/master/nnunetmec2020.ipynb) and the [Medium Article about nnU-Net](https://medium.com/miccai-educational-initiative/nnu-net-the-no-new-unet-for-automatic-segmentation-8d655f3f6d2a)
written in the context of the MICCAI Educational Initiative by:
- Prateek Gupta, Indian Institute of Information Technology, Pune (prateekgupta16@alumni.iiitp.ac.in)
- Kumar T. Rajamani, Institute of Medical Informatics, University of Lübeck, Germany (kumar.rajamani@uni-luebeck.de)
- Mattias P. Heinrich, Institute of Medical Informatics, University of Lübeck, Germany (heinrich@imi.uni-luebeck.de)

Many Thanks to them!

In [ ]:
# Create Folderstructure for the new dataset!
dataset_name = 'Dataset501_SCGM' #change here for different dataset name
nnunet_raw_data = os.path.join(os.getenv("nnUNet_raw"))
task_folder_name = os.path.join(nnunet_raw_data,dataset_name)
train_image_dir = os.path.join(task_folder_name,'imagesTr')
train_label_dir = os.path.join(task_folder_name,'labelsTr')
test_dir = os.path.join(task_folder_name,'imagesTs')

# Create Folder Structure for the SCGM dataset on the system
make_if_dont_exist(task_folder_name)
make_if_dont_exist(train_image_dir)
make_if_dont_exist(train_label_dir)
make_if_dont_exist(test_dir)

training_data_name="training-data-gm-sc-challenge-ismrm16-v20160302b"
test_data_name="test-data-gm-sc-challenge-ismrm16-v20160401"

## Data Download
Again, now the data has to be put into the proper place in the folder structure.

In case you are not attending an in person workshop please skip the next cell

In [ ]:
# only for in poerson workshops

os.chdir(task_folder_name)
# download training data
!gdown 'https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy'
!ls
os.chdir(base_dir)

In [ ]:
# continue from here again.
if os.path.isfile(os.path.join(task_folder_name, training_data_name+'.zip')) is False:
  print("Please download the dataset zipfiles and place them into the following directory: \n {}".format(task_folder_name))
else:
  print(f'Training file for exists SCGM Challenge exists')

In [ ]:
# verify that files are in the correct place!
os.chdir(task_folder_name)
!ls
if os.path.isfile(training_data_name+'.zip'):
    print(f'Training file for exists SCGM Challenge exists')
else:
    print('Training file for SCGM Challenge is not present in the directory')
    print("Please check whether {}.zip is in Folder {}".format(training_data_name, task_folder_name))

os.chdir(base_dir)
print("We are currently in working directory {}".format(os.getcwd()))

In [ ]:
#unzipping in nnUNet_raw folder the training data
os.chdir(task_folder_name)
!unzip -o training-data-gm-sc-challenge-ismrm16-v20160302b.zip
!rm training-data-gm-sc-challenge-ismrm16-v20160302b.zip
os.chdir(base_dir)

## Data Preprocessing
We have 4 annotation of the same image, by different experts in the SCGM Challenge. ( Image , Ann1 ) and ( Image , Ann2 ) can be considered as a different image and label pairs. Hence, 4 copies of the training .nii.gz file is created with its mapping to the respective label name.

For this the data is renamed and relocated

In [ ]:
# function for copying, savind and renaming
def copy_and_rename(old_location,old_file_name,new_location,new_filename,delete_original = False):

    shutil.copy(os.path.join(old_location,old_file_name),new_location)
    os.rename(os.path.join(new_location,old_file_name),os.path.join(new_location,new_filename))
    if delete_original:
        os.remove(os.path.join(old_location,old_file_name))

# putting training images into folder

mask_count = 4 # change if more mask is available
base_data_folder_name = os.path.join(task_folder_name, "training-data-gm-sc-challenge-ismrm16-v20160302b")

for file in os.listdir(base_data_folder_name):
    # print(file)
    if file.endswith('.nii.gz'):
        if file.find('mask')!=-1:
            # putting mask
            shutil.move(os.path.join(base_data_folder_name,file),train_label_dir)
        else:
            # making 4 copies
            for mask in range(1,mask_count+1):
                new_filename = file[:file.find('-image')] + '-mask-r' + str(mask) + '.nii.gz'
                if mask==mask_count:
                    copy_and_rename(base_data_folder_name,file,train_image_dir,new_filename,delete_original = True)
                else:
                    copy_and_rename(base_data_folder_name,file,train_image_dir,new_filename)
    # removing all other files installed due to the unzip
    elif file.endswith('.txt'):
        os.remove(os.path.join(base_data_folder_name,file))

## Verification of Data

Before going any further, verify that the data is present and labels and data matches.

In [ ]:
train_files = os.listdir(train_image_dir)
label_files = os.listdir(train_label_dir)
print("train image files:",len(train_files))
print("train label files:",len(label_files))
print("Matches:",len(set(train_files).intersection(set(label_files))))

assert len(set(train_files).intersection(set(label_files))) == 160 #should be equal to 160 for SCGM Challenge

In [ ]:
#renaming to add the modality for SCGM there is only one modality
#images should be added with 0000
#can be skipped if modality is already mentioned
#re-write for multiple modalities

def check_modality(filename):
    """
    check for the existence of modality
    return False if modality is not found else True
    """
    end = filename.find('.nii.gz')
    modality = filename[end-4:end]
    for mod in modality:
        if not(ord(mod)>=48 and ord(mod)<=57): #if not in 0 to 9 digits
            return False
    return True

def rename_for_single_modality(directory):

    for file in os.listdir(directory):

        if check_modality(file)==False:
            new_name = file[:file.find('.nii.gz')]+"_0000.nii.gz"
            os.rename(os.path.join(directory,file),os.path.join(directory,new_name))
            print(f"Renamed to {new_name}")
        else:
            print(f"Modality present: {file}")

rename_for_single_modality(train_image_dir)

# again skip test due to non available data
# rename_for_single_modality(test_dir)

## Creation of the Dataset501 for the SCGMC Dataset
The Dataset gives nnU-Net Information for the Planning and Preprocessing Step.

Based on the parameters of the dataset, the "optimal strategy" is selected (data fingerprint & pipeline fingerprint)

This is done by creating the ```dataset.json``` file.

In [ ]:
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Dataset_folder
json_file_exist = False

if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:

    json_dict = OrderedDict()
    json_dict['name'] = dataset_name
    json_dict['description'] = "Spinal Cord Grey Matter Segmenation Challenge"
    json_dict['tensorImageSize'] = "3D"
    json_dict['reference'] = "see challenge website"
    json_dict['licence'] = "see challenge website"
    json_dict['release'] = "0.0"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "MRI"
    }

    # set expected file ending
    json_dict["file_ending"] = ".nii.gz"

    #label names should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "grey matter": 1,
        "white matter": 2
    }

    train_ids = os.listdir(train_label_dir)
    test_ids = os.listdir(test_dir)
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTest'] = len(test_ids)


    with open(os.path.join(task_folder_name,"dataset.json"), 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
        if json_file_exist==False:
            print('dataset.json created!')
        else:
            print('dataset.json overwritten!')

## Dataset Preprocessing
Also to ensure that nnU-Net can be trained on the dataset a integrity check is performed.

In [ ]:
# verify that the dataset will work & create plans for configuration 3d_fullres - this may take about 10 minutes
!nnUNetv2_plan_and_preprocess -d 501 -np 4 -c 3d_fullres --verify_dataset_integrity

## Dataset Visualization
At this stage it is useful to examine the training and testing data.

Here is an example for how to do this in python, however we would encourage you to visualize the data with MITK-Workbench, since it allows much more interaction with the data and therefore better understanding.

Train Data (with Labels)

In [ ]:
#visualizing some of the training images and labels
# (re-run to see random pick-ups)
# only maximum of first 5 slices are plotted
train_img_name = os.listdir(train_image_dir)[np.random.randint(0,160)]
train_img = np.array(nib.load(os.path.join(train_image_dir,train_img_name)).dataobj)[:,:,:5]
train_label_name = train_img_name[:train_img_name.find('_0000.nii.gz')]+'.nii.gz'
train_label = np.array(nib.load(os.path.join(train_label_dir,train_label_name)).dataobj)[:,:,:5]

print(train_img.shape,train_label.shape)

max_rows = 2
max_cols = train_img.shape[2]

fig, axes = plt.subplots(nrows=max_rows, ncols=max_cols, figsize=(20,8))
for idx in range(max_cols):
    axes[0, idx].axis("off")
    axes[0, idx].set_title('Train Image'+str(idx+1))
    axes[0 ,idx].imshow(train_img[:,:,idx], cmap="gray")
for idx in range(max_cols):
    axes[1, idx].axis("off")
    axes[1, idx].set_title('Train Label'+str(idx+1))
    axes[1, idx].imshow(train_label[:,:,idx])

plt.subplots_adjust(wspace=.1, hspace=.1)
plt.show()

Note: In the label image, yellow color represents white Matter and green-ish color represents grey matter.

### Sighting of Data and Lables with MITK-Workbench
At this stage it is also useful to analyze the imaging data with MITK-Workbench.
To do this, download the training data to your own local machine and visualize them via drag and drop into MITK-Workbench

This is already explained in part 6.3.4

## Training nnU-Net

nnU-Net stores a checkpoint every 50 epochs. If you need to continue a previous training, just add a -c to the training command.

Generic Training Commands:

```nnUNetv2_train Dataset_NAME_OR_ID CONFIGURATION FOLD -tr TRAINER_CLASS_NAME (additional options)```

For 2D:  ```nnUNetv2_train DATASET_NAME_OR_ID 2d FOLD```

For 3D Full resolution: ```nnUNetv2_train DATASET_NAME_OR_ID 3d_fullres FOLD```

For Cascaded 3D:

First Run lowres: ```nnUNetv2_train DATASET_NAME_OR_ID 3d_lowres FOLD```

Then Run fullres: ```nnUNetv2_train DATASET_NAME_OR_ID 3d_cascade_fullres FOLD```

In [ ]:
# train nnU-Net with 3d_fullres model on the SCGM Task with Fold 0
# Again Training may take a while therefore it is advised to interrupt the training after some time (e.g. after 1 epoch)
# 1 Epoch takes approximately: over 30 minutes on GoogleColab -- you need GoogleColab Pro to run this completely
!nnUNetv2_train 501 3d_fullres 0 -tr nnUNetTrainer_1epoch

## Inference Code

nnUNet_find_best_configuration will print inference commands you need to use. The easiest way to run inference is to simply use these commands.

For each of the desired configurations, run:

```nnUNet_predict -i INPUT_FOLDER -o OUTPUT_FOLDER -d DATASET_NAME_OR_ID -m CONFIGURATION --save_npz```

Only specify ```--save_npz``` if you intend to use ensembling. ```--save_npz``` will make the command save the softmax probabilities alongside of the predicted segmentation masks requiring a lot of disk space.

Note: Please select a separate OUTPUT_FOLDER for each configuration!

In [ ]:
# Optional
!nnUNetv2_find_best_configuration -d 501

In [ ]:
result_dir = os.path.join(os.getenv("RESULTS_FOLDER"),'nnUNet_Prediction_Results',dataset_name)
make_if_dont_exist(result_dir)

team_name = 'awesome_nnU-Net_team' #make sure to change for your own team name

**Note**:

If you interrupted the training go the given fold inside of the RESULTS_FOLDER for the task and:
1. rename **model_best.model.pkl** to **model_final_checkpoint.model.pkl**
2. rename **model_best.model** to **model_final_checkpoint.model**

In [ ]:
#location where you want save your results, will be created if dont exist
!nnUNetv2_predict -i '${nnUNet_raw}/Dataset501_SCGM/imagesTs' -o '${RESULTS_FOLDER}/Dataset501/predTs' -d 501 -tr nnUNetTrainer_1epoch -c 3d_fullres

If you wish to run ensembling, you can ensemble the predictions from several configurations with the following command:

```nnUNet_ensemble -f FOLDER1 FOLDER2 ... -o OUTPUT_FOLDER -pp POSTPROCESSING_FILE```

## Visualization of Predictions
Similar to the dataset visualization we would encourage you to do this with MITIK-Workbench.

Here is a quick visualization with python.

In [ ]:
#visualizing the predicted results
# (re-run to see random pick-ups)
# only maximum of first 5 slices are plotted

test_img_name = os.listdir(test_dir)[np.random.randint(0,40)]
test_img = np.array(nib.load(os.path.join(test_dir,test_img_name)).dataobj)[:,:,:5]
predicted_img_name = test_img_name[:test_img_name.find('_0000.nii.gz')]+'.nii.gz'
predicted_label = np.array(nib.load(os.path.join(result_dir,predicted_img_name)).dataobj)[:,:,:5]
print('Test Image Shape: ',test_img.shape)
print("Predicted Image Shape:",predicted_label.shape)

max_rows = 2
max_cols = test_img.shape[2]

fig, axes = plt.subplots(nrows=max_rows, ncols=max_cols, figsize=(20,8))
for idx in range(max_cols):
    axes[0, idx].axis("off")
    axes[0, idx].set_title('Test Image'+str(idx+1))
    axes[0 ,idx].imshow(test_img[:,:,idx], cmap="gray")
for idx in range(max_cols):
    axes[1, idx].axis("off")
    axes[1, idx].set_title('Predicted Label'+str(idx+1))
    axes[1, idx].imshow(predicted_label[:,:,idx])

plt.subplots_adjust(wspace=.1, hspace=.1)
plt.show()